## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
from tqdm import tqdm
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    

with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        foods.append(food)
        users.append(user)
        pbar.update(1)
        

        
food_map = {f:i for i, f in enumerate(set(foods))}
user_map = {u:i for i, u in enumerate(set(users))}



100%|██████████| 2681494/2681494 [00:20<00:00, 133456.28it/s]


In [2]:
import numpy as np
a = []
b = []

for k,v in u_map.items():
    ds = [d for d,f in v]
    
    set_ds = set(ds)
    a.append(len(set_ds))
    b.append(len(ds))
print max(a), min(a),np.mean(a), np.std(a)
print max(b), min(b),np.mean(b), np.std(b)

160 23 95.21165644171779 34.48409857182605
5569 194 1028.1802147239264 599.6642539977294


### Generate training data and label

In [3]:
### ALS embeddingimport numpy as  np
from scipy.sparse import csr_matrix

with tqdm(total=len(u_map)) as pbar:
    
#     split 1/10 for future
    Y_map = {}
    pairs = []
    pseudo_history_map = {} # for training and validation
    for user,v in u_map.items():
        history = v
        dates_history = [d for d,f in history]
        min_date, max_date = min(dates_history), max(dates_history)
        date_list = sorted(set(dates_history))
        rest_date_set = set(date_list[-7:])
        
        
        pseudo_history_map[user] = [(d,f) for d,f in history if d not in rest_date_set]
#         calculate the Y
        y = np.zeros([1, len(food_map)])
        for d,f in history:
            if d in rest_date_set:
                y[0, food_map[f]] = 1
            
            else:
                pair = (user_map[user], food_map[f])
                pairs.append(pair)
        Y_map[user] = y
        pbar.update(1)
        
    rows = []
    cols = []
    pairs = set(pairs)
    for pair in pairs:
        u, f = pair
        rows.append(u)
        cols.append(f)
    data = np.ones([len(rows),])
    R = csr_matrix((data, (rows,cols)), shape=(len(user_map),len(food_map)))
        
    
print len(Y_map), R.shape
        
    



100%|██████████| 2608/2608 [00:01<00:00, 1339.23it/s]

2608 (2608, 5532)


### ALS embedding Training

In [4]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=32, use_gpu=True, iterations=100)
user_items = R.transpose()
model.fit(user_items)
print model.user_factors.shape
print R.shape


100%|██████████| 100.0/100 [00:00<00:00, 208.71it/s]

(2608, 32)
(2608, 5532)


### Transformer with ALS embedding Training

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import Transformer
import numpy as np
from constants import FOOD_NUM
class Net(nn.Module):

    def __init__(self, dm, p_drop):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p_drop)
        self.food_emb = Food_embedding(FOOD_NUM, dm, 2, p_drop)
        self.target_food_emb = nn.Embedding(FOOD_NUM, dm)
        self.summary_weight = nn.Parameter(torch.FloatTensor(1, dm))
        nn.init.xavier_normal_(self.summary_weight)
        self.linears = nn.ModuleList([nn.Linear(2*dm, 2*dm) for i in range(2)])
        self.output_linear = nn.Linear(2*dm, 1)

    def forward(self, food, history):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        batch, _ =  food.size() 
        target = self.target_food_emb(food).view(batch, -1)
        
        K = history
        
        K = self.food_emb(K)
        
        
        batch, K_len, d = K.size()
        Q = self.summary_weight.repeat(batch,1).view(batch, 1, -1)
        att_out = self.attention(Q, K)
        x = torch.cat([att_out, target],dim=-1)
        for lay in self.linears:
            x = F.selu(lay(x))
            x = self.drop(x)
        x = self.output_linear(x)
        y = torch.sigmoid(x)
        return y
    def to_ont_hot(self, one_hot_dim, indices):
        batch, _ = indices.size()
        one_hot = torch.zeros(batch, one_hot_dim, requires_grad=False)
        one_hot.scatter_(1, indices.cpu(), 1)
        return one_hot.cuda()
    def attention(self, Q,K):
        assert len(Q.shape) == 3 and len(K.shape) == 3
        e = torch.bmm(Q,K.permute(0,2,1))
        
        att = F.softmax(e, dim=-1)
        out = torch.bmm(att, K).squeeze(1)
        return out
        
class Food_embedding(nn.Module):
    def __init__(self, c_in, dm, layer_num, p_drop, activation_fn=F.selu):
        super(Food_embedding, self).__init__()
        self.activation_fn = activation_fn
        self.drop = nn.Dropout(p_drop)
        assert layer_num >= 1
        self.first_linear = nn.Linear(c_in, dm)
        self.linears = nn.ModuleList([nn.Linear(dm, dm) for i in range(layer_num-1)])
        

    def forward(self, x):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        x = self.first_linear(x)
        x = self.activation_fn(x)
        x = self.drop(x)
        for lay in self.linears:
            x = self.drop(self.activation_fn(lay(x)))
        return x
    
batch = 7
dm = 128
K = torch.rand([batch, 18, FOOD_NUM]).cuda()
Q = torch.randint(FOOD_NUM,[batch,1], dtype=torch.long).cuda()
model = Net(dm, 0.1).cuda()
print(Q.dtype)
o = model(Q, K)
# print t
print(o.size())
# print o

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))


criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

torch.int64
torch.Size([7, 1])
1564801


#### prepare sample generator


In [6]:
from constants import MAX_SEQ_LEN

val_num = len(pseudo_history_map) // 10
idx = np.random.permutation(len(pseudo_history_map))
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:pseudo_history_map[k] for k in pseudo_history_map.keys()[val_num:]}
val_u_map = {k:pseudo_history_map[k] for k in pseudo_history_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, u_map, food_map, Y_map, max_history_len, flip):
    G = boostrap_generator(u_map, food_map, Y_map, max_history_len, flip)
    while True:
        X = []
        pos_Y = []
        neg_Y = []
        for i in range(batch_size):
            x, pos_y, neg_y = next(G)
            x = np.pad(x, ((0,max_history_len-x.shape[0]),(0,0)), 'constant', constant_values=0)
            X.append(np.expand_dims(x, axis=0))
            pos_Y.append(np.expand_dims(pos_y, axis=0))
            neg_Y.append(np.expand_dims(neg_y, axis=0))
        yield np.vstack(X), np.vstack(pos_Y), np.vstack(neg_Y) 
def boostrap_generator(u_map, food_map, Y_map, max_history_len, flip):
    while True:
        keys = u_map.keys()
        for user_idx in np.random.permutation(len(u_map)):
            user = keys[user_idx]
            X = np.zeros([max_history_len, len(food_map)])
            Y = Y_map[user].flatten()
            history = u_map[user]
            ds = np.array([d for d,f in history])
            fs = np.array([f for d,f in history])
            sorted_idx = np.flip(np.argsort(ds), axis=-1) if flip else np.argsort(ds)
            ds = ds[sorted_idx]
            fd = fs[sorted_idx]
            
            date_idx = 0
            now_date = ds[0]
            for food, date in zip(fs,ds):
                if date != now_date:
                    date_idx+=1
                    now_date = date
                X[date_idx, food_map[food]] = 1
            
    #         positive sample
            idx = np.random.permutation(len(Y))
            pos_i = neg_i = -1
            for i in idx:
                if Y[i] == 1 and pos_i == -1:
                    pos_i = i
                if Y[i] == 0 and neg_i == -1:
                    neg_i = i
                if pos_i != -1 and neg_i != -1:
                    break
            yield X, np.array([pos_i]), np.array([neg_i])
            
    

G = batch_boostrap_generator(32, train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=True)
val_G = batch_boostrap_generator(32//2, val_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=True)

x, pos_y, neg_y = next(G)
print x.shape, pos_y.shape, neg_y.shape
x, pos_y, neg_y = next(val_G)
print x.shape, pos_y.shape, neg_y.shape

G2 = boostrap_generator(train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=True)
x, pos_y, neg_y = next(G2)
print x.shape, pos_y.shape, neg_y.shape


(32, 165, 5532) (32, 1) (32, 1)
(16, 165, 5532) (16, 1) (16, 1)
(165, 5532) (1,) (1,)


In [7]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
    log_file_stream.write(log_text)
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
        
acc_q = deque(maxlen=1000)
loss_q = deque(maxlen=1000)
val_acc_q = deque(maxlen=1000)
val_loss_q = deque(maxlen=1000)
t = time.time()
best_acc  = 0
best_loss = float('inf')

epochs = 100
batch_size = 64
G = batch_boostrap_generator(batch_size, train_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=False)
val_G = batch_boostrap_generator(batch_size, val_u_map, food_map, Y_map, max_history_len=MAX_SEQ_LEN, flip=False)

print 'start training.'
with open('log_att.txt', 'a') as f:
    with open('best_att.txt', 'w') as best_log:
        for e in range(epochs):
            iters = len(food_map) * len(user_map)
            with tqdm(total=iters) as pbar:
                for it in range(iters):
                    optimizer.zero_grad()
                    model.train()
                    x, pos_y, neg_y = next(G)
    #                 print 'x1', np.sum(x[0,:])
                    x = torch.FloatTensor(x).cuda()
                    pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)

    #                 positive
                    output = model(pos_y.cuda(), x)
                    a = output[0,0].item()
                    pred = output > 0.5
                    label = torch.ones_like(output).cuda()
                    pos_loss = criterion(output, label)

                    acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                    acc_q.append(acc)
    #                 pos_loss.backward()

    #                 negative
                    output = model(neg_y.cuda(), x)
                    b = output[0,0].item()
                    pred = output > 0.5
                    label = torch.zeros_like(output).cuda()
                    neg_loss = criterion(output, label )
                    acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                    acc_q.append(acc)
    #                 neg_loss.backward()

                    loss = pos_loss + neg_loss
                    loss.backward()
                    optimizer.step()
                    with torch.no_grad():
                        model.eval()
    #                 positive
                        x, pos_y, neg_y = next(val_G)
    #                     print 'x2', np.sum(x[0,:])
                        x = torch.FloatTensor(x).cuda()
                        pos_y,neg_y = torch.LongTensor(pos_y), torch.LongTensor(neg_y)
    #                       
                        output = model(pos_y.cuda(), x)
    #                     o1 = model.transformer.decoder.mid
    #                     o1 = model.mid

                        c = output[0,0].item()
                        pred = output > 0.5
                        label = torch.ones_like(output).cuda()

                        pos_loss = criterion(output, label )

                        val_acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                        val_acc_q.append(val_acc)
    #                 negative
    #                     print pos_y, neg_y
    #                     print output

                        output = model(neg_y.cuda(), x)
    #                     o2 = model.mid
    #                     print o2.shape
    #                     print neg_y.cuda().shape
    #                     print neg_y.cuda()[0,0]
    #                     print (o1 == o2).all()
                        d = output[0,0].item()
    #                     print c == d
    #                     wf
    #                     print pos_y == neg_y
    #                     print output


                        pred = output > 0.5
                        label = torch.zeros_like(output).cuda()

                        neg_loss = criterion(output, label )
                        val_acc = torch.sum(pred == label.type(torch.uint8)).item() / float(output.shape[0])
                        val_acc_q.append(val_acc)

                        val_loss = pos_loss + neg_loss
                    acc = np.mean(acc_q)
                    val_acc = np.mean(val_acc_q)

    #                 dump_log(model, it+1, loss, acc, val_loss, val_acc, f, './tmp.pt')

    #                 pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, %.3f, %.3f, %.3f' % (acc, val_acc,a,b,c), refresh=False)
                    pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, %.3f, %.3f, %.5f, %.5f' % (acc, val_acc,a,b,c,d), refresh=False)
                    pbar.update(batch_size)
                    dump_log(model, (it+1)*batch_size, loss, acc, val_loss, val_acc, f, './tmp_att.pt')
                    if val_acc > best_acc and it > 100:
                        torch.save(model, './best_att.pt')
                        best_acc = val_acc
                        best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_acc))

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

start training.

  0%|          | 0/14427456 [00:00<?, ?it/s]

  0%|          | 256/14427456 [00:04<74:13:04, 54.00it/s, acc : 0.488, val_acc : 0.471, 0.422, 0.464, 0.47487, 0.52471]


KeyboardInterrupt: 

### testing with Transformer model

In [ ]:
from constants import MAX_SEQ_LEN


        
flip = False       
model = torch.load('./best.pt')
model.eval()
batch_size = 256
food_buf = torch.LongTensor(np.arange(len(food_map))).view(1,-1).repeat(batch_size, 1).cuda()
answer_sheet = np.zeros([len(u_map), len(food_map)])

with torch.no_grad():
    with tqdm(total=len(u_map)*len(food_map)) as pbar:
        keys = u_map.keys()
        ran = range((len(keys) // batch_size) +1) if len(keys) % batch_size != 0 else range(len(keys) // batch_size)
        for i in ran:
            a = i*batch_size
            b = (i+1)*batch_size if (i+1)*batch_size < len(keys) else len(keys)
            X = []
            for user in keys[a:b]:
                x = np.zeros([MAX_SEQ_LEN, len(food_map)])
                history = u_map[user]
                ds = np.array([d for d,f in history])
                fs = np.array([f for d,f in history])
                sorted_idx = np.flip(np.argsort(ds), axis=-1) if flip else np.argsort(ds)
                ds = ds[sorted_idx]
                fd = fs[sorted_idx]

                date_idx = 0
                now_date = ds[0]
                for food, date in zip(fs,ds):
                    if date != now_date:
                        date_idx+=1
                        now_date = date
                    x[date_idx, food_map[food]] = 1
                X.append(np.expand_dims(x,axis=0))
            
            X = torch.FloatTensor(np.vstack(X)).cuda()
            for food_idx  in range(len(food_map)):
                target = food_buf[:b-a, food_idx:food_idx+1]
                output = model(target, X)
                answer_sheet[a:b, food_idx:food_idx+1] =  output.cpu()
                pbar.update(batch_size)
                
np.save('./output_sheet', answer_sheet)
print answer_sheet.shape
print'Done'

rev_food_map = {v:k for k,v in food_map.items()}
k=20
a = ''
buf = []
with open('predict.csv', 'w') as f:
    f.write('userid,foodid\n')
    for i,user in enumerate(u_map.keys()):
        s = ''
        for food_idx in reversed(np.argsort(answer_sheet[i,:])[-k:]):
            s += ' %d' % rev_food_map[food_idx]
        f.write('%d,%s\n' % (user, s) )
        buf.append(a == s)
        a = s
print buf
print 'done'

    
            


In [ ]:
m = {user:i for i,user in enumerate(u_map.keys())}
for k in u_map.keys():
    assert m[k] == user_map[k]
